In [ ]:
import os
import sys
sys.path.append('/Users/guns/Documents/GitHub/storage_python')  # my_apikeys.py 파일이 있는 경로
import my_apikeys as mykeys
import urllib.request
import pandas as pd
import json

# 네이버에서 발급받은 클라이언트 ID와 시크릿을 사용
client_id = mykeys.client_id
client_secret = mykeys.client_secret

# 파라미터 설정
display_count = 100     # 한 페이지에 표시할 검색 결과 수
num_data = 1000         # 검색할 데이터 개수
sort = 'date'           # 정렬 기준 (date: 날짜순, sim: 유사도순)

# 검색할 단어와 URL 설정
encText = urllib.parse.quote("서울시 부동산")

# 결과를 저장할 list 생성
results = []

# for문을 사용하여 검색 결과를 페이지별로 요청
for idx in range(1, num_data + 1, display_count):

    # JSON 결과 요청 URL 생성
    url = (
        "https://openapi.naver.com/v1/search/news?query="
        + encText
        + f"&start={idx}&display={display_count}&sort={sort}"
    )

    # 요청 객체 생성
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    # 요청 보내서 응답 받기
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:  # 응답 코드가 200이면 성공
        # 응답 본문을 읽음
        response_body = response.read()
        # response_body는 바이트 문자열이므로 decode를 통해 문자열로 변환
        response_dict = json.loads(response_body.decode('utf-8'))
        # dictionary에서 'items' 키를 사용하여 뉴스 기사 목록을 가져옴
        results = results + response_dict['items']
    else:
        print("Error Code:", rescode)

# 데이터 개수 확인
print(f"총 데이터 개수: {len(results)}")

# 일부 데이터 출력
results[:3]

총 데이터 개수: 1000


[{'title': "이재명·오세훈 호평에 뜨는 정원오…'다윗과 골리앗 싸움' 조짐",
  'originallink': 'https://news.tf.co.kr/read/life/2272335.htm',
  'link': 'https://n.news.naver.com/mnews/article/629/0000453147?sid=102',
  'description': "야권 내부에서는 '현역 프리미엄'으로 <b>서울시</b>장 선거를 두고 여전히 승산이 있다는 인식이 나온다. <b>부동산</b> 대책에 따른 서울 민심이 이반이 큰 데다, 현직 시장인 오 시장이 갖는 인지도와 조직력 역시 무시하기 어렵다는... ",
  'pubDate': 'Mon, 15 Dec 2025 00:01:00 +0900'},
 {'title': '정원오 <b>서울시</b>장 출마 선언 임박? “서울을 글로벌 G2 메가시티로 만들...',
  'originallink': 'https://slownews.kr/150852',
  'link': 'https://slownews.kr/150852',
  'description': '정원오가 생각하는 ‘서울의 비전’은?<b>서울시</b>장에 출마한다면 공약은?구청장 11년 동안 기억에 남는 사건을... <b>부동산</b>도 많이 올랐다. 아파트 매매 지수가 2014년 대비 두 배가 넘는다. 상승률 기준으로 4위다. 윤석열 3년... ',
  'pubDate': 'Sun, 14 Dec 2025 21:40:00 +0900'},
 {'title': '[스트레이트] 누구를 위한 사업인가',
  'originallink': '',
  'link': 'https://n.news.naver.com/mnews/article/214/0001467996?sid=102',
  'description': '[이광수/<b>부동산</b> 전문가] &quot;기업이 그렇게 (땅을) 사는 건 흔치 않은 경우라고 저는 보고 있고 아, 이건 확신이... 세 차례에 걸쳐 <b>서

In [2]:
from datetime import datetime
import re

# 저장할 빈 데이터프레임 생성
df = pd.DataFrame()

# 문자열에서 제거할 tag를 지정
remove_tags = re.compile(r'<.*?>')  # HTML 태그 제거를 위한 정규표현식

# 검색 결과에서 필요한 정보 추출
for item in results:
    # 각 뉴스 기사에서 필요한 정보 추출
    new_data = pd.DataFrame(
        data={
            # 날짜는 datetime 객체로 변환
            'pubDate': datetime.strptime(
                item['pubDate'],
                "%a, %d %b %Y %H:%M:%S +0900"
            ),
            'title': re.sub(remove_tags, '', item['title']),
            'description': re.sub(remove_tags, '', item['description'])
        },
        index=[0]  # index를 0으로 설정하여 단일 행 데이터프레임 생성
    )

    # 데이터프레임에 추가
    df = pd.concat([df, new_data], ignore_index=True)

# 데이터프레임 출력
df.head()

,pubDate,title,description
0,2025-12-15 00:01:00,이재명·오세훈 호평에 뜨는 정원오…'다윗과 골리앗 싸움' 조짐,야권 내부에서는 '현역 프리미엄'으로 서울시장 선거를 두고 여전히 승산이 있다는 인...
1,2025-12-14 21:40:00,정원오 서울시장 출마 선언 임박? “서울을 글로벌 G2 메가시티로 만들...,정원오가 생각하는 ‘서울의 비전’은?서울시장에 출마한다면 공약은?구청장 11년 동안...
2,2025-12-14 21:14:00,[스트레이트] 누구를 위한 사업인가,[이광수/부동산 전문가] &quot;기업이 그렇게 (땅을) 사는 건 흔치 않은 경우...
3,2025-12-14 19:38:00,"오세훈 &quot;정부, 10.15 대책 부작용 외면…규제 완화하라&quot;",&quot;실수요자 투기꾼 취급하는 대출 정책도 즉각 전환해야&quot; 오세훈 서...
4,2025-12-14 19:32:00,오세훈 &quot;10.15 부동산대책은 악정(惡政)&quot;,"&quot;정부, 주거 불안 해소는커녕 오히려 부추긴 꼴&quot; 오세훈 서울시장..."


In [3]:
from datetime import datetime
import re

# 저장할 빈 데이터프레임 생성
df = pd.DataFrame()

# 문자열에서 제거할 tag를 지정
remove_tags = re.compile(r'<.*?>')  # HTML 태그 제거를 위한 정규표현식

# 검색 결과에서 필요한 정보 추출
for item in results:
    # 각 뉴스 기사에서 필요한 정보 추출
    new_data = pd.DataFrame(
        data={
            # 날짜는 datetime 객체로 변환
            'pubDate': datetime.strptime(item['pubDate'], "%a, %d %b %Y %H:%M:%S +0900"),
            # title과 description에서 HTML 태그 제거
            'title': re.sub(remove_tags, '', item['title']),
            'description': re.sub(remove_tags, '', item['description'])
        },
        index=[0]  # index를 0으로 설정하여 단일 행 데이터프레임 생성
    )

    # 데이터프레임에 추가
    df = pd.concat([df, new_data], ignore_index=True)

# 데이터프레임 출력
df.head()

,pubDate,title,description
0,2025-12-15 00:01:00,이재명·오세훈 호평에 뜨는 정원오…'다윗과 골리앗 싸움' 조짐,야권 내부에서는 '현역 프리미엄'으로 서울시장 선거를 두고 여전히 승산이 있다는 인...
1,2025-12-14 21:40:00,정원오 서울시장 출마 선언 임박? “서울을 글로벌 G2 메가시티로 만들...,정원오가 생각하는 ‘서울의 비전’은?서울시장에 출마한다면 공약은?구청장 11년 동안...
2,2025-12-14 21:14:00,[스트레이트] 누구를 위한 사업인가,[이광수/부동산 전문가] &quot;기업이 그렇게 (땅을) 사는 건 흔치 않은 경우...
3,2025-12-14 19:38:00,"오세훈 &quot;정부, 10.15 대책 부작용 외면…규제 완화하라&quot;",&quot;실수요자 투기꾼 취급하는 대출 정책도 즉각 전환해야&quot; 오세훈 서...
4,2025-12-14 19:32:00,오세훈 &quot;10.15 부동산대책은 악정(惡政)&quot;,"&quot;정부, 주거 불안 해소는커녕 오히려 부추긴 꼴&quot; 오세훈 서울시장..."


In [4]:
df.to_csv(
    "/Users/guns/Desktop/naver_news.csv",
    index=False,
    encoding="utf-8-sig"
)